In [72]:
!hdfs dfs -rm -R /h1
!hdfs dfs -rm -R /h2

rm: `/h1': No such file or directory
rm: `/h2': No such file or directory


In [1]:
!hdfs dfs -rm -R /h1
!hdfs dfs -rm -R /h2
!hadoop fs -mkdir -p /h1
!hadoop fs -put data2/testbfs.txt /h1

Deleted /h1
Deleted /h2
2020-10-25 04:40:48,035 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [3]:
!hadoop dfs -ls /h1


Found 1 items
-rw-r--r--   3 root supergroup        172 2020-10-25 04:40 /h1/testbfs.txt


In [5]:
%%writefile mapper.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*
import sys
import re

black_count = 0
for line in sys.stdin: # reads from stdin
    line = line.strip()
    if("GRAY" in line):
        black_line = line.replace("GRAY","BLACK")
        print(black_line)
        print("reporter:counter:%s,%s,%d" % ('example-info','black_count',1), file=sys.stderr)
        source, info = line.split()
        kids,score,color,parent = info.split("|")
        score = int(score)+1
        kids = kids.split(",")
        for kid in kids:
            print(kid+"\tnull|"+str(score)+"|GRAY|"+source)
    else:
        print(line)
        if("BLACK" in line):
            print("reporter:counter:%s,%s,%d" % ('example-info','black_count',1), file=sys.stderr)    
        


Writing mapper.py


In [7]:
%%writefile reducer.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys
import random

node = dict()
for line in sys.stdin:
    line = line.strip()
#     print(line)
    source,info=line.split()

# #     if("5" in source):
    kids,score,color,parent = info.split("|")
    if(not node): #fill node if currently empty
        node['source'] = source
        node['kids'] = kids
        node['score'] = score
        node['color'] = color
        node['parent'] = parent
    else:
        score_updated = False
            #dont need to touch the source field anymore. partitioner already groups by source field
            #only replace kids if find node copy with kids
        if(kids != "null"):
            node['kids'] = kids
            
#         if("null" not in kids):
#             node['kids'] = kids
            #ignore max value lines

        if(score.isnumeric()):
            if(node['score'] == "Integer.MAX_VALUE"):
                node['score'] = score
                score_updated = True
            elif(int(node['score']) > int(score)):
                node['score'] = score
                score_updated = True
#         if("Integer.MAX_VALUE" not in score):
#                 #check if a lower score found. Replace if true
#             if(int(node['score']) > int(score)):
#                 node['score'] = score
            
            #take darkest color
        if("WHITE" == node['color'] and "WHITE" != color):#if color is darker than white, replace
            node['color'] = color
        elif("GRAY" == node['color'] and "BLACK" == color):#if color is darker than gray, replace
            node['color'] = color
                
            #if incoming parent is null or source do nothing, parent needs to be a number to compare
            # also only change parent if score was changed.

            
        if(parent == 'source'):
            node['parent'] = 'source'
            break
        if(score_updated == True):
            if(node['parent'] == 'null' and parent.isnumeric()):
                node['parent'] = parent
            elif(node['parent'].isnumeric() and parent.isnumeric()):
                if(int(node['parent']) > int(parent)):
                    node['parent'] = parent




print(node['source']+"\t"+node['kids']+"|"+node['score']+"|"+node['color']+"|"+node['parent'])

Overwriting reducer.py


# Testing

In [9]:
!cat data2/testbfs.txt | python mapper.py 

1	2,3|0|BLACK|source
reporter:counter:example-info,black_count,1
2	null|1|GRAY|1
3	null|1|GRAY|1
2	1,3,4,5|Integer.MAX_VALUE|WHITE|null
3	1,4,2|Integer.MAX_VALUE|WHITE|null
4	2,3|Integer.MAX_VALUE|WHITE|null
5	2|Integer.MAX_VALUE|WHITE|null


In [62]:
!hdfs dfs -rm -R /outputs
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D mapreduce.job.reduces=5 \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data2/int3.txt \
    -output /outputs/out > /dev/null 2> err.txt \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

Deleted /outputs


In [85]:
!cat err.txt

/dev/null


In [86]:
!hdfs dfs -ls /outputs/out

Found 6 items
-rw-r--r--   3 root supergroup          0 2020-10-23 00:20 /outputs/out/_SUCCESS
-rw-r--r--   3 root supergroup         20 2020-10-23 00:19 /outputs/out/part-00000
-rw-r--r--   3 root supergroup         18 2020-10-23 00:20 /outputs/out/part-00001
-rw-r--r--   3 root supergroup         16 2020-10-23 00:20 /outputs/out/part-00002
-rw-r--r--   3 root supergroup         14 2020-10-23 00:20 /outputs/out/part-00003
-rw-r--r--   3 root supergroup         21 2020-10-23 00:20 /outputs/out/part-00004


In [57]:
!hdfs dfs -cat /outputs/result2/part-00000
!hdfs dfs -cat /outputs/result2/part-00001
!hdfs dfs -cat /outputs/result2/part-00002
!hdfs dfs -cat /outputs/result2/part-00003
!hdfs dfs -cat /outputs/result2/part-00004

2020-10-22 23:46:11,770 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2	1,3,4,5|1|BLACK|1
2020-10-22 23:46:16,014 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
3	1,4,2|1|BLACK|1
2020-10-22 23:46:20,498 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
4	2,3|2|BLACK|2
2020-10-22 23:46:24,194 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
5	2|2|BLACK|2
2020-10-22 23:46:28,783 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
1	2,3|0|BLACK|source


In [58]:
!hdfs dfs -cat /outputs/result2/part-00000  /outputs/result2/part-00001 /outputs/result2/part-00002 /outputs/result2/part-00003 /outputs/result2/part-00004  

2020-10-22 23:53:26,836 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2	1,3,4,5|1|BLACK|1
3	1,4,2|1|BLACK|1
4	2,3|2|BLACK|2
5	2|2|BLACK|2
1	2,3|0|BLACK|source


# Final Output

In [10]:
%%writefile script.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*
import os

black_count=0
i=1
o=2
while(black_count<5):
    print("black_count="+str(black_count))
    print("interation="+str(i))
#     black_count=5#black_count+1
    cmd1 = 'hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D mapreduce.job.reduces=5 \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /h{} \
    -output /h{} > /dev/null 2> err.txt \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner'.format(str(i),str(o))   
    os.system(cmd1)
    word = "black_count"
    with open("err.txt") as search:
        for line in search:
            line = line.rstrip()  # remove '\n' at end of line
            if word in line:
                black_count = int(line[-1])
    i = i + 1
    o = o + 1


Writing script.py


In [20]:
!python script.py

black_count=0
interation=1
black_count=1
interation=2
black_count=3
interation=3


In [22]:
!hadoop dfs -ls /h4


Found 6 items
-rw-r--r--   3 root supergroup          0 2020-10-25 05:24 /h4/_SUCCESS
-rw-r--r--   3 root supergroup         20 2020-10-25 05:23 /h4/part-00000
-rw-r--r--   3 root supergroup         18 2020-10-25 05:24 /h4/part-00001
-rw-r--r--   3 root supergroup         16 2020-10-25 05:24 /h4/part-00002
-rw-r--r--   3 root supergroup         14 2020-10-25 05:24 /h4/part-00003
-rw-r--r--   3 root supergroup         21 2020-10-25 05:24 /h4/part-00004


In [23]:
!hadoop dfs -cat /h4/part-00000
!hadoop dfs -cat /h4/part-00001
!hadoop dfs -cat /h4/part-00002
!hadoop dfs -cat /h4/part-00003
!hadoop dfs -cat /h4/part-00004


2020-10-25 05:32:44,710 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2	1,3,4,5|1|BLACK|1

2020-10-25 05:32:48,666 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
3	1,4,2|1|BLACK|1

2020-10-25 05:32:52,419 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
4	2,3|2|BLACK|2

2020-10-25 05:32:58,107 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
5	2|2|BLACK|2

2020-10-25 05:33:01,977 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
1	2,3|0|BLACK|source
